### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [17]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,\
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum(((l_c ** 2 / l_s) + (r_c ** 2 / r_s)) / (l_s + r_s),
                          axis=1)# Ваш код в 1 строчку
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum((np.log2(l_c / l_s) * l_c +
                        np.log2(r_c / r_s) * r_c) / (l_s + r_s), axis=1)# Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max((l_c + r_c) / (l_s + r_s), axis=1)# Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.sqrt(n_feature))])# Ваш код в 1 строчку
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.log2(n_feature))])# Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)# Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Ваш код
        x_sort, y_sort = self.__sort_samples(x, y)

        if type(self.min_samples_split) is int:
            tmp = self.min_samples_split
        else:  # float
            tmp = int(ceil(self.min_samples_split * x_sort.shape[0]))
        min_size = np.int(tmp / 2 - 1)

        if min_size:
            y_split = y_sort[min_size * 2:]
        else:
            y_split = y_sort
        border_ids = np.where(y_split[1:] != y_split[:-1])[0] + min_size*2 + 1

        if border_ids.size == 0:
            return np.inf, None

        class_count = border_ids - \
            np.append(np.array([min_size * 2]), border_ids[:-1])
        number_of_classes = np.zeros((border_ids.shape[0], self.num_class))
        number_of_classes[np.arange(border_ids.shape[0]),
                          y_sort[border_ids - 1]] = class_count

        l_class = np.cumsum(number_of_classes, axis=0)
        r_class = np.bincount(y_sort,
                              minlength=self.num_class) - l_class
        l_size = border_ids.reshape(l_class.shape[0], 1)
        r_size = y_sort.shape[0] - l_size

        G = self.G_function(l_class, l_size, r_class, r_size)
        i = np.argmin(G)
        return G[i], (x_sort[border_ids[i] - 1] + x_sort[border_ids[i]]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        # Ваш код
        nums_class = np.bincount(y)
        max_num_class = np.argmax(nums_class)
        if y.shape[0] < self.min_samples_split or\
                np.max(nums_class) / y.size >= self.sufficient_share or\
                self.max_depth and depth >= self.max_depth:
            self.tree[node_id] = (self.LEAF_TYPE,
                                  max_num_class, nums_class / y.size)
            return

        features_id = self.get_feature_ids(x.shape[1])
        all_thresholds = np.array([self.__find_threshold(x[:, i], y)
                                   for i in features_id])

        best_feature_id = all_thresholds[:, 0].argmin()
        best_threshold = all_thresholds[best_feature_id, 1]
        if not best_threshold:
            self.tree[node_id] = (self.LEAF_TYPE,
                                  max_num_class, nums_class / y.size)
            return
        x_left, x_right, y_left, y_right =\
            self.__div_samples(x, y,
                               features_id[best_feature_id],
                               best_threshold)

        if x_left.size == 0 or x_right.size == 0:
            self.tree[node_id] = (self.LEAF_TYPE, max_num_class,
                                  nums_class.astype(np.float) / y.size)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              features_id[best_feature_id],
                              best_threshold)
        self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [6]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [7]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data,\
                                                    wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 1.07 ms, sys: 3.29 ms, total: 4.35 ms
Wall time: 286 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 10.9 ms, sys: 8.82 ms, total: 19.7 ms
Wall time: 116 ms


## Проверка качества работы на wine

In [10]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8374727668845315

In [11]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [19]:
#%%pycodestyle


df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position',
              'positin1', 'order', 'partner', 'age_o', 'race_o',
              'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun',
              'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o',
              'sinc_o', 'intel_o', 'fun_o', 'amb_o',
              'shar_o', 'like_o', 'prob_o', 'met_o'], axis=1)

df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df.loc[:, 'field_cd']
# pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'tuition'] =\
    df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining',
              'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater',
              'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)

'''
attr 1
'''
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1',
        'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                        'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1', 'shar1_1']] = (
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
               'fun1_1', 'amb1_1', 'shar1_1']].T /
    df.loc[:, 'temp_totalsum'].T).T * 100

'''
Attr 2
'''
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1',
        'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = (
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
               'fun2_1', 'amb2_1', 'shar2_1']].T /
    df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)
'''
'''
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
'''
'''
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
    .drop(['gender'], axis=1).dropna()

df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'

'''
Join
'''
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
features_names = df_pair.columns[1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = DecisionTreeClassifier(min_samples_split=2)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [13]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

CPU times: user 78.3 ms, sys: 2.12 ms, total: 80.4 ms
Wall time: 125 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

CPU times: user 286 ms, sys: 3.95 ms, total: 290 ms
Wall time: 291 ms


## Проверка качества работы на Speed Dating Data

In [15]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5535714285714286

In [16]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.4594594594594595

## Задание 3

## Задание 4

## Задание 5